In [1]:
#include <iostream>
#include <thread>
#include <vector>
#include <algorithm>

### scoped_thread的用法

In [2]:
namespace n1 {

    void do_something(unsigned i)
    {
        std::cout << "do_something: i = " << i << "\n";
    }

    struct func
    {
        int& i;
        func(int& i_) : i(i_) {}
        void operator() () {
            for (unsigned j = 0; j < 3; ++j) {
                do_something(i);
            }
        }
    };

    class scoped_thread
    {
        std::thread t;
    public:
        explicit scoped_thread(std::thread t_) : // 1
            t(std::move(t_))
        {
            if (!t.joinable()) // 2
                throw std::logic_error("No thread");
        }
        ~scoped_thread()
        {
            t.join(); // 3
        }
        scoped_thread(scoped_thread const&) = delete;
        scoped_thread& operator=(scoped_thread const&) = delete;
    };
}

In [3]:
{
    using namespace n1;

    int some_local_state = 1;
    func my_func(some_local_state);
    std::thread t(my_func);
    t.join();
}

do_something: i = 1
do_something: i = 1
do_something: i = 1


In [4]:
{
    using namespace n1;
    
    int some_local_state = 1;
    func my_func(some_local_state);
    scoped_thread st{std::thread(my_func)};
}

do_something: i = 1
do_something: i = 1
do_something: i = 1


In [5]:
{
    using namespace n1;
    
    int some_local_state = 1;
    scoped_thread st(std::thread(func(some_local_state)));
}

input_line_7:6:21: warning: parentheses were disambiguated as a function declaration [-Wvexing-parse]
    scoped_thread st(std::thread(func(some_local_state)));
                    ^~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
input_line_7:6:22: note: add a pair of parentheses to declare a variable
    scoped_thread st(std::thread(func(some_local_state)));
                     ^
                     (                                  )


In [6]:
{
    using namespace n1;
    
    int some_local_state = 1;
    scoped_thread st{std::thread(func(some_local_state))};
}

do_something: i = 1
do_something: i = 1
do_something: i = 1


### 量产线程，等待它们结束

In [7]:
namespace n2 {
    void do_work(unsigned id)
    {
        std::cout << "do_word( " << id << " )\n";
    }
}

In [8]:
{
    using namespace n2;

    std::vector<std::thread> threads;
    for (unsigned i = 0; i < 5; ++i) {
        threads.push_back(std::thread(do_work, i)); // 产生线程
    }
    std::for_each(threads.begin(), threads.end(),
        std::mem_fn(&std::thread::join)); // 对每个线程调用join()
}

do_word( 0 )
do_word( 1 )
do_word( 2 )
do_word( 3 )
do_word( 4 )
